In [1]:
import pandas as pd
import numpy as np

In [2]:

df_train = pd.read_csv("round1_ijcai_18_train_20180301.csv", sep=' ')
 
df_test = pd.read_csv("round1_ijcai_18_test_a_20180301.csv", sep=' ')

In [3]:
df = df_train

In [4]:
print("类别数量统计")
for name in df.head(0):
    print(name," "*(30-len(name)),len(df[name].unique()))

类别数量统计
instance_id                     478087
item_id                         10075
item_category_list              14
item_property_list              10908
item_brand_id                   2055
item_city_id                    128
item_price_level                14
item_sales_level                18
item_collected_level            18
item_pv_level                   22
user_id                         197694
user_gender_id                  4
user_age_level                  9
user_occupation_id              5
user_star_level                 12
context_id                      478111
context_timestamp               281824
context_page_id                 20
predict_category_property       78796
shop_id                         3959
shop_review_num_level           25
shop_review_positive_rate       11825
shop_star_level                 22
shop_score_service              16361
shop_score_delivery             16423
shop_score_description          16463
is_trade                        2


In [5]:
"""
input:
sr1: pandas serise
sr2: pandas serise
return :a list of the intersection of sr1 and sr2

"""

def cross_check(sr1,sr2): 

    a = sr1.unique()
    b = sr2.unique()
    #print(len(a))
    relist = list(set(a).intersection(set(b)))
    checkpass = len(relist) == len(a)
    return relist,checkpass


len(cross_check(df_test["item_id"],df_train["item_id"]))

2

In [6]:
print("缺失值统计")
for name in df.head(0):
    
    if -1 in df[name].value_counts():
        print(name," "*(30-len(name)),df[name].value_counts()[-1])

缺失值统计
item_brand_id                   473
item_city_id                    277
item_sales_level                913
user_gender_id                  12902
user_age_level                  964
user_occupation_id              964
user_star_level                 964
shop_review_positive_rate       7
shop_score_service              59
shop_score_delivery             59
shop_score_description          59


In [7]:
feature2process = [name for name in df.head(0)]
feature2drop = []

# 全部待处理变量
```
'instance_id',
 'item_id',
 'item_category_list',
 'item_property_list',
 'item_brand_id',
 'item_city_id',
 'item_price_level',
 'item_sales_level',
 'item_collected_level',
 'item_pv_level',
 'user_id',
 'user_gender_id',
 'user_age_level',
 'user_occupation_id',
 'user_star_level',
 'context_id',
 'context_timestamp',
 'context_page_id',
 'predict_category_property',
 'shop_id',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_star_level',
 'shop_score_service',
 'shop_score_delivery',
 'shop_score_description',
 'is_trade'
```

## 处理instance变量 ，主要完成去重部分; [0]

In [55]:
print(feature2process[0])
print("去重前")
print(df_train.count()[0])
print(df_test.count()[0])

df_train_0 = df_train.drop_duplicates([feature2process[0]])
df_test_0 = df_test.drop_duplicates([feature2process[0]])

print("去重后")
print(df_train_0.count()[0])
print(df_test_0.count()[0])

instance_id
去重前
478138
18371
去重后
478087
18371


## 处理item_id变量   ;1
**对item_id 进行onehot处理，仅保留大于1000的类目，其余都归为other类，并增加count类,对于NaN则全部置零,如无特殊情况之后删除item_id列**



In [48]:
def onehot_with_dict(sr,dummy_dict = None,count_dict = None,add_count = True):
    if  dummy_dict is None:
        dummy_dict = list(sr.unique())
    if -1 in dummy_dict:
        dummy_dict.remove(-1)
    lines = []
    line_length = len(dummy_dict) + 1
    if add_count and  count_dict is None:
        count_dict = sr.value_counts()
        count_dict[-1] = 0
    for cat_name in sr:
        new_line = [0.0] * line_length
        if cat_name in dummy_dict:
            new_line[dummy_dict.index(cat_name)] = 1
        elif cat_name != -1:
            new_line[line_length - 1] = 1
            
        if add_count and cat_name in count_dict:
            new_line.append(count_dict[cat_name])
        elif add_count :
            new_line.append(0)
        lines.append(new_line)
    header = [str(sr.name)+ '_' +str(cat_name) for cat_name in dummy_dict] 
    header.append((str(sr.name)+ '_' +"other"))
    header.append((str(sr.name)+ '_' +"count"))
    
    return pd.DataFrame(np.array(lines),index = sr.index , columns = header),count_dict



In [59]:
def onehot_wrapper(df1,df2,sr1,sr2,dummy_dict = None):
    onehot_sr1, count = onehot_with_dict(sr1,dummy_dict=dummy_dict)
    onehot_sr2 ,_     = onehot_with_dict(sr2,dummy_dict=dummy_dict, count_dict=count)
    return df1.join(onehot_sr1), df2.join(onehot_sr2)
 

In [60]:
this_feature = feature2process[1]
sr_test = df_test_0[this_feature]
sr_train = df_train_0[this_feature]
common_id,_ = cross_check(sr_test,sr_train)

temp = df_train[this_feature].value_counts()[common_id]
print(temp.describe())
threshold = 1000
temp = temp[temp > threshold]
print(len(temp))
dummy_dict = list(temp.index)
#pd.get_dummies(df[feature],prefix = dummy_dict)
"""
df_train_p = df_train_p.drop(this_feature, axis = 1)
df_test_p = df_test_p.drop(this_feature, axis = 1)
"""
feature2drop.append(this_feature)

df_train_1, df_test_1 = onehot_wrapper(df_train_0,df_test_0,sr_train,sr_test,dummy_dict = dummy_dict)



count    3534.000000
mean      119.605263
std       227.768686
min         1.000000
25%        14.000000
50%        44.000000
75%       121.000000
max      3001.000000
Name: item_id, dtype: float64
45


In [61]:
print(df_train_1.shape)
print(df_test_1.shape)

(478087, 74)
(18371, 73)


## 处理item_category_list ;2

可以看出所有商品的cat0是相同的，可以直接抛弃

只有子类目不同只需要对子类目进行one-hot

所有商品都有cat1，直接进行one_hot编码

部分商品有cat2，有cat2的正常进行one_hot编码，对于没有的置为（0，0）。

In [63]:
def get_sub_cat(sr):
    a = []
    b = []
    c = []
    i = 0
    for cat in sr:
        a.append(cat.split(';')[0])
        b.append(cat.split(';')[1])
        if len(cat.split(';')) != 2:
            c.append(cat.split(';')[2])
        else:
            c.append(None)
        #print(i,cat) 
        #i += 1
    cat0 = pd.Series(a,index = sr.index)
    cat1 = pd.Series(b,index = sr.index)
    cat2 = pd.Series(c,index = sr.index)
    return cat0, cat1, cat2

In [64]:
this_feature = feature2process[2]
sr_test = df_test_1[this_feature]
sr_train = df_train_1[this_feature]

_, cat_train1, cat_train2 = get_sub_cat(sr_train)
_, cat_test1, cat_test2 = get_sub_cat(sr_test)
print(cross_check(cat_train1,cat_test1)[1])
print(cross_check(cat_train2,cat_test2)[1])

df_train_2 = df_train_1.join(pd.get_dummies(cat_train1,prefix = 'item_cat1' ).astype('float'))
df_train_2 = df_train_2.join(pd.get_dummies(cat_train2,prefix = 'item_cat2' ).astype('float'))

df_test_2 = df_test_1.join(pd.get_dummies(cat_train1,prefix = 'item_cat1' ).astype('float'))
df_test_2 = df_test_2.join(pd.get_dummies(cat_train2,prefix = 'item_cat2' ).astype('float'))


feature2drop.append(this_feature)

#pd.get_dummies(cat_train2).astype('float')
#cat_train2
print(df_train_2.shape)
print(df_test_2.shape)

True
True
(478087, 89)
(18371, 88)


# 处理item_property_list；3

经过测试所有的每个纪录之间的property数量相差很大

In [65]:
this_feature = feature2process[3]
sr_test = df_test_2[this_feature]
sr_train = df_train_2[this_feature]


In [66]:
length = []
for cat in sr_train:
    length.append(len(cat.split(';')))
print(set(length))

length = []
for cat in sr_test:
    length.append(len(cat.split(';')))
print(set(length))

{6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 82, 85, 88, 89, 91, 93, 97, 99, 100}
{7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 74, 75, 76, 77, 82, 85, 88, 89, 91, 97, 99, 100}


In [67]:
def get_property_dict(sr):
    property_dict = {}
    for cat in sr:
        for item_property in cat.split(';'):
            if item_property not in property_dict:
                property_dict[item_property] = 1
            else:
                property_dict[item_property] += 1
    return property_dict

In [68]:
print(len(get_property_dict(sr_train)))
print(len(get_property_dict(sr_test)))
df_test_3 = df_test_2 
df_train_3 = df_train_2

61407
26261


# 处理item_brand_id；4

与item_id 类似，仅保留大于2000的类目，其余都归为other类，并增加count类,对于NaN则全部置零,如无特殊情况之后删除item_id列

In [69]:
this_feature = feature2process[4]
print(this_feature)
sr_test = df_test_3[this_feature]
sr_train = df_train_3[this_feature]
common_id,_ = cross_check(sr_test,sr_train)



item_brand_id


In [70]:
temp = df_train[this_feature].value_counts()[common_id]
print(temp.describe())
threshold = 2000
temp = temp[temp > threshold]
print(len(temp))
dummy_dict = list(temp.index)

feature2drop.append(this_feature)

df_train_4, df_test_4 = onehot_wrapper(df_train_3,df_test_3,sr_train,sr_test,dummy_dict = dummy_dict)

count     1081.000000
mean       431.760407
std       2290.940261
min          1.000000
25%         29.000000
50%        106.000000
75%        343.000000
max      69746.000000
Name: item_brand_id, dtype: float64
38


# 处理item_city_id；5

与item_id 类似，仅保留大于2000的类目，其余都归为other类，并增加count类,对于NaN则全部置零,如无特殊情况之后删除item_id列

In [71]:
this_feature = feature2process[5]
print(this_feature)
sr_test = df_test_4[this_feature]
sr_train = df_train_4[this_feature]
common_id,b = cross_check(sr_test,sr_train)
print(b)



item_city_id
True


In [72]:
temp = df_train[this_feature].value_counts()[common_id]
print(temp.describe())
threshold = 10000
temp = temp[temp > threshold]
print(len(temp))

dummy_dict = list(temp.index)

feature2drop.append(this_feature)

df_train_5, df_test_5 = onehot_wrapper(df_train_4,df_test_4,sr_train,sr_test,dummy_dict = dummy_dict)

count        99.000000
mean       4825.222222
std       18039.425273
min           2.000000
25%          88.500000
50%         277.000000
75%        1466.500000
max      154991.000000
Name: item_city_id, dtype: float64
9


# 处理item_price_level ；6

# 处理item_sales_level ；7

# 处理item_collected_level ；8

# 处理item_pv_level；9

# 处理user_id；10

In [102]:
"""
 把一个sr依照count_dict变为计数编码，如果不在countdict中或者为缺失值，则置0
"""

def cat2count(sr,count_dict = None):
    sr.value_counts()
    lines = []
    if count_dict is None:
        count_dict = sr.value_counts()
        count_dict[-1] = 0
    for cat_name in sr:
        if cat_name in count_dict:
            new_line = [count_dict[cat_name]]
        else:
            new_line = [0]
        lines.append(new_line)
    header = [(str(sr.name)+ '_' +"count")]
    return pd.DataFrame(np.array(lines),index = sr.index , columns = header) ,count_dict




"""
sr= pd.Series([1,2,4,1,2,-1,-1,2,4,0,10])
a,count = cat2count(sr)
print(a)
sr= pd.Series([1,2,1,2,8,5,9,2,4,0,-1,-1])
b,_ = cat2count(sr,count)
print(b)
"""

'\nsr= pd.Series([1,2,4,1,2,-1,-1,2,4,0,10])\na,count = cat2count(sr)\nprint(a)\nsr= pd.Series([1,2,1,2,8,5,9,2,4,0,-1,-1])\nb,_ = cat2count(sr,count)\nprint(b)\n'

In [92]:
"""
此处均为按照train集的计数来进行变换。
"""

def count_wrapper(df1,df2,sr1,sr2):
    a,count = cat2count(sr1)
    b,_ = cat2count(sr2,count_dict=count)
    return df1.join(a),df2.join(b)

In [93]:
this_feature = feature2process[10]
print(this_feature)
sr_test = df_test_5[this_feature]
sr_train = df_train_5[this_feature]
common_id,b = cross_check(sr_test,sr_train)
print(b)

feature2drop.append(this_feature)

df_train_10, df_test_10 = count_wrapper(df_train_5,df_test_5,sr_train,sr_test)


user_id
False


In [95]:
len(common_id)

3626

In [100]:
sr_test.value_counts()[common_id].describe()

count    3626.000000
mean        1.426641
std         0.879653
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        14.000000
Name: user_id, dtype: float64

In [101]:
sr_train.value_counts()[common_id].describe()

count    3626.000000
mean        3.736624
std         4.132611
min         1.000000
25%         1.000000
50%         2.000000
75%         5.000000
max        56.000000
Name: user_id, dtype: float64

In [110]:
# 交易成功次数最多的id都只有四次
temp = df_train.is_trade
sr_train[temp == 1].value_counts().describe()

count    8778.000000
mean        1.024607
std         0.160708
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         4.000000
Name: user_id, dtype: float64

# 处理user_gender_id；11

In [111]:
this_feature = feature2process[11]
print(this_feature)
sr_test = df_test_10[this_feature]
sr_train = df_train_10[this_feature]

user_gender_id


In [114]:
df_train_11 = df_train_10.join(pd.get_dummies(sr_train,prefix = 'user_gender_id' ).astype('float'))
df_test_11 = df_test_10.join(pd.get_dummies(sr_test,prefix = 'user_gender_id' ).astype('float'))

# 处理user_age_level；12

# 处理user_occupation_id ；13

In [117]:
this_feature = feature2process[13]
print(this_feature)
sr_test = df_test_11[this_feature]
sr_train = df_train_11[this_feature]

user_occupation_id


In [118]:
df_train_13 = df_train_11.join(pd.get_dummies(sr_train,prefix = 'user_occupation_id' ).astype('float'))
df_test_13 = df_test_11.join(pd.get_dummies(sr_test,prefix = 'user_occupation_id' ).astype('float'))

# 处理user_star_level； 14